<a href="https://colab.research.google.com/github/aml7hawaiiedu/CCAPLandCoverProject/blob/main/CCAP_UNET_processing_Fall2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import shutil
import hashlib

In [ ]:
# # OR Unzipping to a specific directory
!mkdir /content/CCAP

In [ ]:
!mkdir '/content/Hawaii_2005_003'
!mkdir '/content/Hawaii_2005_003/Images'
!mkdir '/content/Hawaii_2005_003/Labels'

In [ ]:
!mkdir '/content/Hawaii_2005_004'
!mkdir '/content/Hawaii_2005_004/Images'
!mkdir '/content/Hawaii_2005_004/Labels'

In [ ]:
!mkdir '/content/Hawaii_2005_005'
!mkdir '/content/Hawaii_2005_005/Images'
!mkdir '/content/Hawaii_2005_005/Labels'

mkdir: cannot create directory ‘/content/Hawaii_2005_005’: File exists
mkdir: cannot create directory ‘/content/Hawaii_2005_005/Images’: File exists
mkdir: cannot create directory ‘/content/Hawaii_2005_005/Labels’: File exists


In [ ]:
!mkdir '/content/Hawaii_2005_006'
!mkdir '/content/Hawaii_2005_006/Images'
!mkdir '/content/Hawaii_2005_006/Labels'

mkdir: cannot create directory ‘/content/Hawaii_2005_006’: File exists
mkdir: cannot create directory ‘/content/Hawaii_2005_006/Images’: File exists
mkdir: cannot create directory ‘/content/Hawaii_2005_006/Labels’: File exists


In [ ]:
!mkdir '/content/Hawaii_2005_007'
!mkdir '/content/Hawaii_2005_007/Images'
!mkdir '/content/Hawaii_2005_007/Labels'

In [ ]:
!mkdir '/content/Hawaii_2005_008'
!mkdir '/content/Hawaii_2005_008/Images'
!mkdir '/content/Hawaii_2005_008/Labels'

In [ ]:
!mkdir '/content/Hawaii_2005_009'
!mkdir '/content/Hawaii_2005_009/Images'
!mkdir '/content/Hawaii_2005_009/Labels'

In [41]:
# Unzip the file and place the files to the newly cerated directory (CCAP/image_chips/ISLAND)
!unzip /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/revised_trainingdata/Lanai.zip -d /content/CCAP/

Archive:  /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/revised_trainingdata/Lanai.zip
   creating: /content/CCAP/content/Lanai/
  inflating: /content/CCAP/content/Lanai/2005_Lanai_CCAP.tif  
  inflating: /content/CCAP/content/Lanai/2005_Lanai_LS.tif  
   creating: /content/CCAP/content/Lanai/Images/
  inflating: /content/CCAP/content/Lanai/Images/1_1.tif  
  inflating: /content/CCAP/content/Lanai/Images/1_1025.tif  
  inflating: /content/CCAP/content/Lanai/Images/1_1537.tif  
  inflating: /content/CCAP/content/Lanai/Images/1_2049.tif  
  inflating: /content/CCAP/content/Lanai/Images/1_2561.tif  
  inflating: /content/CCAP/content/Lanai/Images/1_3073.tif  
  inflating: /content/CCAP/content/Lanai/Images/1_3585.tif  
  inflating: /content/CCAP/content/Lanai/Images/1_4097.tif  
  inflating: /content/CCAP/content/Lanai/Images/1_4609.tif  
  inflating: /content/CCAP/content/Lanai/Images/1_5121.tif  
  inflating: /content/CCAP/content/Lanai/Images/1_513.tif  
  inflating: /content/CC

Next two blocks of code are probably unnecessary but run them for a sanity check.

In [42]:
!mkdir '/content/Lanai'
!mkdir '/content/Lanai/Images'
!mkdir '/content/Lanai/Labels'

In [43]:
# original images and labels folder: how many files are there?
org_img = '/content/CCAP/content/Lanai/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')
org_label = '/content/CCAP/content/Lanai/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

The number of files in the folder is: 437
The number of files in the folder is: 437


In [44]:
df = pd.read_csv('/content/CCAP/content/Lanai/Lanai.csv')

In [45]:
df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent
0,513,513,1,1,1,700998.805448,702227.628989,2.314891e+06,2.316120e+06,0,1.000000
1,1025,513,513,1,2,702227.596419,703456.375946,2.314891e+06,2.316120e+06,0,1.000000
2,1537,513,1025,1,3,703456.405579,704685.211254,2.314891e+06,2.316120e+06,0,0.889732
3,1537,513,1025,1,3,703456.405579,704685.211254,2.314891e+06,2.316120e+06,10,0.000351
4,1537,513,1025,1,3,703456.405579,704685.211254,2.314891e+06,2.316120e+06,12,0.000130
...,...,...,...,...,...,...,...,...,...,...,...
1693,9729,9497,9217,8985,433,723117.205661,724346.018919,2.293330e+06,2.294558e+06,0,1.000000
1694,10241,9497,9729,8985,434,724345.986782,725574.791585,2.293330e+06,2.294558e+06,0,1.000000
1695,10753,9497,10241,8985,435,725574.784059,726803.598098,2.293330e+06,2.294558e+06,0,1.000000
1696,11265,9497,10753,8985,436,726803.616834,728032.402301,2.293330e+06,2.294558e+06,0,1.000000


In [46]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [47]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent
99,1537,1025,1025,513,26,703456.408765,704685.207789,2.313662e+06,2.314891e+06,15,0.000828


In [48]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [49]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent,tif_name
99,1537,1025,1025,513,26,703456.408765,704685.207789,2.313662e+06,2.314891e+06,15,0.000828,1025_513.tif


In [50]:
# Copy the useable images files to new folder
source_images = '/content/CCAP/content/Lanai/Images'
destination_images = '/content/Lanai/Images'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

Copied 1025_513.tif to /content/Lanai/Images


In [51]:
# Copy the useable labels files to new folder
source_labels = '/content/CCAP/content/Lanai/Labels'
destination_labels = '/content/Lanai/Labels'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

Copied 1025_513.tif to /content/Lanai/Labels


In [52]:
# NEW images and labels folder: how many files are there?
new_img = '/content/Lanai/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')
new_label = '/content/Lanai/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

The number of files in the folder is: 1
The number of files in the folder is: 1


In [53]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/Lanai_subset.csv'
subset_df.to_csv(csv_file_path, index=False)
print(f"DataFrame exported to {csv_file_path}")

DataFrame exported to /content/Lanai_subset.csv


In [55]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/Lanai'
zip_file_path = '/content/Lanai_subset.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)
print(f'ZIP file created: {zip_file_path}')

ZIP file created: /content/Lanai_subset.zip


In [57]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata'
shutil.move(zip_file_path, drive_folder)
print(f'ZIP file moved to Google Drive: {drive_folder}')
shutil.move(csv_file_path, drive_folder)
print(f'CSV file moved to Google Drive: {drive_folder}')

ZIP file moved to Google Drive: /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata
CSV file moved to Google Drive: /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata


# Define next folder

In [ ]:
# Unzip the file and place the files to the newly cerated directory (CCAP/image_chips/ISLAND)
!unzip /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/Hawaii_Island_Update/Export/Hawaii_2005_008.zip -d /content/CCAP/

Archive:  /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/Hawaii_Island_Update/Export/Hawaii_2005_008.zip
   creating: /content/CCAP/content/Hawaii_2005_008/Images/
  inflating: /content/CCAP/content/Hawaii_2005_008/Images/1_1.tif  
  inflating: /content/CCAP/content/Hawaii_2005_008/Images/1_10241.tif  
  inflating: /content/CCAP/content/Hawaii_2005_008/Images/1_1025.tif  
  inflating: /content/CCAP/content/Hawaii_2005_008/Images/1_10753.tif  
  inflating: /content/CCAP/content/Hawaii_2005_008/Images/1_11265.tif  
  inflating: /content/CCAP/content/Hawaii_2005_008/Images/1_11777.tif  
  inflating: /content/CCAP/content/Hawaii_2005_008/Images/1_12289.tif  
  inflating: /content/CCAP/content/Hawaii_2005_008/Images/1_12801.tif  
  inflating: /content/CCAP/content/Hawaii_2005_008/Images/1_13313.tif  
  inflating: /content/CCAP/content/Hawaii_2005_008/Images/1_13825.tif  
  inflating: /content/CCAP/content/Hawaii_2005_008/Images/1_14337.tif  
  inflating: /content/CCAP/content/Hawaii_2

In [ ]:
# original images and labels folder: how many files are there?
org_img = '/content/CCAP/content/Hawaii_2005_008/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')
org_label = '/content/CCAP/content/Hawaii_2005_008/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

The number of files in the folder is: 1476
The number of files in the folder is: 1476


In [ ]:
df = pd.read_csv('/content/CCAP/content/Hawaii_2005_008/Hawaii_2005_008.csv')

In [ ]:
df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent
0,513,513,1,1,1,266696.349253,267925.153036,2.192373e+06,2.193601e+06,10,0.040760
1,513,513,1,1,1,266696.349253,267925.153036,2.192373e+06,2.193601e+06,13,0.565292
2,513,513,1,1,1,266696.349253,267925.153036,2.192373e+06,2.193601e+06,14,0.378246
3,513,513,1,1,1,266696.349253,267925.153036,2.192373e+06,2.193601e+06,15,0.014881
4,513,513,1,1,1,266696.349253,267925.153036,2.192373e+06,2.193601e+06,21,0.000820
...,...,...,...,...,...,...,...,...,...,...,...
5934,16385,20982,15873,20470,1472,304789.146854,306017.937229,2.143247e+06,2.144476e+06,0,1.000000
5935,16897,20982,16385,20470,1473,306017.906862,307246.710622,2.143247e+06,2.144476e+06,0,1.000000
5936,17409,20982,16897,20470,1474,307246.786944,308475.556486,2.143247e+06,2.144476e+06,0,1.000000
5937,17921,20982,17409,20470,1475,308475.521470,309704.303091,2.143247e+06,2.144476e+06,0,1.000000


In [ ]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [ ]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [ ]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent,tif_name
1,513,513,1,1,1,266696.349253,267925.153036,2.192373e+06,2.193601e+06,13,0.565292,1_1.tif
2,513,513,1,1,1,266696.349253,267925.153036,2.192373e+06,2.193601e+06,14,0.378246,1_1.tif
3,513,513,1,1,1,266696.349253,267925.153036,2.192373e+06,2.193601e+06,15,0.014881,1_1.tif
6,1025,513,513,1,2,267925.107689,269153.923863,2.192373e+06,2.193601e+06,13,0.387817,513_1.tif
7,1025,513,513,1,2,267925.107689,269153.923863,2.192373e+06,2.193601e+06,14,0.126968,513_1.tif
...,...,...,...,...,...,...,...,...,...,...,...,...
4445,3585,15873,3073,15361,1087,274069.134657,275297.924941,2.155509e+06,2.156737e+06,15,0.000507,3073_15361.tif
4791,2049,16897,1537,16385,1156,270382.716590,271611.509194,2.153051e+06,2.154280e+06,15,0.000847,1537_16385.tif
4949,17409,16897,16897,16385,1186,307246.716513,308475.514444,2.153051e+06,2.154280e+06,18,0.000889,16897_16385.tif
5121,16897,17409,16385,16897,1221,306017.958385,307246.752157,2.151822e+06,2.153051e+06,16,0.002216,16385_16897.tif


In [ ]:
# Copy the useable images files to new folder
source_images = '/content/CCAP/content/Hawaii_2005_008/Images'
destination_images = '/content/Hawaii_2005_008/Images'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

In [ ]:
# Copy the useable labels files to new folder
source_labels = '/content/CCAP/content/Hawaii_2005_008/Labels'
destination_labels = '/content/Hawaii_2005_008/Labels'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

In [ ]:
# NEW images and labels folder: how many files are there?
new_img = '/content/Hawaii_2005_008/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')
new_label = '/content/Hawaii_2005_008/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

The number of files in the folder is: 72
The number of files in the folder is: 72


In [ ]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/Hawaii_2005_008_subset.csv'
subset_df.to_csv(csv_file_path, index=False)
print(f"DataFrame exported to {csv_file_path}")

DataFrame exported to /content/Hawaii_2005_008_subset.csv


In [ ]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/Hawaii_2005_008'
zip_file_path = '/content/Hawaii_2005_008_subset.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)
print(f'ZIP file created: {zip_file_path}')

ZIP file created: /content/Hawaii_2005_008_subset.zip


In [ ]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata'
shutil.move(zip_file_path, drive_folder)
print(f'ZIP file moved to Google Drive: {drive_folder}')
shutil.move(csv_file_path, drive_folder)
print(f'CSV file moved to Google Drive: {drive_folder}')

ZIP file moved to Google Drive: /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata
CSV file moved to Google Drive: /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata


# one more to do 7, 8, 9 together

In [ ]:
# Unzip the file and place the files to the newly cerated directory (CCAP/image_chips/ISLAND)
!unzip /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/Hawaii_Island_Update/Export/Hawaii_2005_009.zip -d /content/CCAP/

Archive:  /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/Hawaii_Island_Update/Export/Hawaii_2005_009.zip
   creating: /content/CCAP/content/Hawaii_2005_009/Images/
  inflating: /content/CCAP/content/Hawaii_2005_009/Images/1_1.tif  
  inflating: /content/CCAP/content/Hawaii_2005_009/Images/1_10241.tif  
  inflating: /content/CCAP/content/Hawaii_2005_009/Images/1_1025.tif  
  inflating: /content/CCAP/content/Hawaii_2005_009/Images/1_10753.tif  
  inflating: /content/CCAP/content/Hawaii_2005_009/Images/1_11265.tif  
  inflating: /content/CCAP/content/Hawaii_2005_009/Images/1_11777.tif  
  inflating: /content/CCAP/content/Hawaii_2005_009/Images/1_12289.tif  
  inflating: /content/CCAP/content/Hawaii_2005_009/Images/1_12801.tif  
  inflating: /content/CCAP/content/Hawaii_2005_009/Images/1_13313.tif  
  inflating: /content/CCAP/content/Hawaii_2005_009/Images/1_13825.tif  
  inflating: /content/CCAP/content/Hawaii_2005_009/Images/1_14337.tif  
  inflating: /content/CCAP/content/Hawaii_2

In [ ]:
# original images and labels folder: how many files are there?
org_img = '/content/CCAP/content/Hawaii_2005_009/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')
org_label = '/content/CCAP/content/Hawaii_2005_009/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

The number of files in the folder is: 1476
The number of files in the folder is: 1476


In [ ]:
df = pd.read_csv('/content/CCAP/content/Hawaii_2005_009/Hawaii_2005_009.csv')

In [ ]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [ ]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [ ]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent,tif_name


In [ ]:
# Copy the useable images files to new folder
source_images = '/content/CCAP/content/Hawaii_2005_009/Images'
destination_images = '/content/Hawaii_2005_009/Images'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

In [ ]:
# Copy the useable labels files to new folder
source_labels = '/content/CCAP/content/Hawaii_2005_009/Labels'
destination_labels = '/content/Hawaii_2005_009/Labels'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

In [ ]:
# NEW images and labels folder: how many files are there?
new_img = '/content/Hawaii_2005_009/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')
new_label = '/content/Hawaii_2005_009/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

The number of files in the folder is: 0
The number of files in the folder is: 0


In [ ]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/Hawaii_2005_009_subset.csv'
subset_df.to_csv(csv_file_path, index=False)
print(f"DataFrame exported to {csv_file_path}")

DataFrame exported to /content/Hawaii_2005_009_subset.csv


In [ ]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/Hawaii_2005_009'
zip_file_path = '/content/Hawaii_2005_009_subset.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)
print(f'ZIP file created: {zip_file_path}')

ZIP file created: /content/Hawaii_2005_009_subset.zip


In [ ]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata'
shutil.move(zip_file_path, drive_folder)
print(f'ZIP file moved to Google Drive: {drive_folder}')
shutil.move(csv_file_path, drive_folder)
print(f'CSV file moved to Google Drive: {drive_folder}')

ZIP file moved to Google Drive: /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata
CSV file moved to Google Drive: /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata
